libraries

In [ ]:
import numpy as np
import tensorflow as tf
# import tensorflow_hub as hub

Save Models

In [ ]:
saved_model_path = "Saved_Model_With_TF1"
tf.saved_model.save(model, saved_model_path)
model.save("MobileNetWithoutTheLast6Layers.h5")
print("Saved Model")
saved_model_path = "Saved_Model_With_TFWith_Keras"
tf.keras.models.save_model(model, saved_model_path)

Load Models

In [ ]:
loaded_model = tf.saved_model.load("0_Saved_Model_With_TF1")
loaded_model_keras = tf.keras.models.load_model("0_Saved_Model_With_TFWith_Keras")
loaded_model_keras_h5 = tf.keras.models.load_model('0_MobileNetWithoutTheLast6Layers.h5', custom_objects={'KerasLayer':hub.KerasLayer})
loaded_Pruned_model_keras_h5 = tf.keras.models.load_model('MobileNetWithoutTheLast9Layers.h5', custom_objects={'KerasLayer':hub.KerasLayer})

save weights

In [ ]:
loaded_model_keras_h5.save_weights("PretrainedWeights.h5")

Build dataSet from Directory

In [ ]:
data_dir = 'D:/kolya/4th year/GP/New Idea/GitHub Codes/Sign-Language-Translator-main/ASL Alphabitic/asl_alphabet_train/asl_alphabet_train'

pixels = 224
IMAGE_SIZE = (pixels, pixels)
print(f"Input size {IMAGE_SIZE}")

BATCH_SIZE = 16#@param {type:"integer"}

normalization_layer = tf.keras.layers.Rescaling(1. / 255)
preprocessing_model = tf.keras.Sequential([normalization_layer])

def build_dataset(subset):
  return tf.keras.preprocessing.image_dataset_from_directory(
      data_dir,
      validation_split=.20,
      subset=subset,
      label_mode="categorical",
      # Seed needs to provided when using validation_split and shuffle = True.
      # A fixed seed is used so that the validation set is stable across runs.
      seed=123,
      image_size=IMAGE_SIZE,
      batch_size=1)

msh fahm hna leh b3ml kol dah 
fahm 7tt el normalization layer bs

In [ ]:
normalization_layer = tf.keras.layers.Rescaling(1. / 255)
preprocessing_model = tf.keras.Sequential([normalization_layer])

train_ds = build_dataset("training")
class_names = tuple(train_ds.class_names)
train_size = train_ds.cardinality().numpy()
train_ds = train_ds.unbatch().batch(BATCH_SIZE)
train_ds = train_ds.repeat()

train_ds = train_ds.map(lambda images, labels:
                        (preprocessing_model(images), labels))

val_ds = build_dataset("validation")
valid_size = val_ds.cardinality().numpy()
val_ds = val_ds.unbatch().batch(BATCH_SIZE)
val_ds = val_ds.map(lambda images, labels:
                    (normalization_layer(images), labels))

Evalute the model on the dataset

In [ ]:
loaded_Pruned_model_keras_h5.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

loaded_Pruned_model_keras_h5.evaluate(val_ds,verbose = 1)

tare2a tania 3shan a3ml load l el dataset 

bst5dm m3aha .fit_generator msh .fit bs 

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
train_data_dir = 'D:/kolya/4th year/GP/New Idea/GitHub Codes/Sign-Language-Translator-main/ASL Alphabitic/asl_alphabet_train/asl_alphabet_train'


train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

# this is the augmentation configuration we will use for testing:
# only rescaling
test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(224, 224),
    batch_size=1,
    class_mode='categorical')

In [ ]:
import tqdm
import numpy as np
train_generator.reset()
batch_size = 1
X_train, y_train = next(train_generator)
for i in (range(int(len(train_generator)/batch_size)-1)): #1st batch is already fetched before the for loop.
  img, label = next(train_generator)
  X_train = np.append(X_train, img, axis=0 )
  y_train = np.append(y_train, label, axis=0)
print(X_train.shape, y_train.shape)

tare2a a5od beha el DataSet bardo tania

In [ ]:
import os
import cv2 
import numpy as np
path = 'D:/kolya/4th year/GP/New Idea/GitHub Codes/Sign-Language-Translator-main/ASL Alphabitic/asl_alphabet_train/asl_alphabet_train'
data = []
label = []

label_one_hot_encodded = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
labels_one_hot_encodded =[]

Files = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 
           'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 
           'W', 'X', 'Y', 'Z', 'del', 'nothing', 'space']
label_val = 0
for files in Files:
    cpath = os.path.join(path, files)
    cpath = os.path.join(cpath)
    print(cpath)
    label_one_hot_encodded[label_val] = 1
    for img in os.listdir(cpath):
        image_array = cv2.imread(os.path.join(cpath, img), cv2.IMREAD_COLOR)
        image_array = cv2.resize(image_array,(224,224))
        # image_array = np.array(image_array)/255
        data.append(image_array)
        label.append(label_val)

        labels_one_hot_encodded.append(label_one_hot_encodded.copy()) 


    label_one_hot_encodded[label_val] = 0 #reseting to the old vector
    label_val = label_val + 1


from sklearn.model_selection import train_test_split
# data = np.asarray(data)
# label = np.asarray(label)

X_train, X_test, y_train, y_test = train_test_split(data, label, test_size=0.33, random_state=42)

Predicting an image

In [ ]:
model = tf.keras.models.load_model('MobileNetWithoutTheLast6Layers.h5', custom_objects={'KerasLayer':hub.KerasLayer})
import cv2
classes = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 
           'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 
           'W', 'X', 'Y', 'Z', 'del', 'nothing', 'space']

word2 = cv2.imread("test_forC.jpeg")
word2 = cv2.resize(word2,(224,224))     # resize image to match model's expected sizing


def preprocess_Input(X):
    np_X = np.array(X)
    normalised_X = np_X.astype('float32')/255.0
    return normalised_X

word2 = preprocess_Input(word2)



prediction_scores = model.predict(np.expand_dims(word2, axis=0))
predicted_index = np.argmax(prediction_scores)
print(prediction_scores)
print("Predicted label: " + classes[predicted_index])

Predicting list of images

In [ ]:
import tensorflow_hub as hub
import tensorflow as tf
import cv2
import os

classes = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 
           'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 
           'W', 'X', 'Y', 'Z', 'del', 'nothing', 'space']

path = 'Test Images'
# model = tf.keras.models.load_model('words_model_kaggle.h5', custom_objects={'KerasLayer':hub.KerasLayer})

for i in range(0,29) :
    currentImage = classes[i]+"_test.jpg"
    currentImage = os.path.join(path, currentImage)
    word2 = cv2.imread(currentImage)
    # word2 = cv2.resize(word2,(200,200))     # resize image to match model's expected sizing

    def preprocess_Input(X):
        np_X = np.array(X)
        normalised_X = np_X.astype('float32')/255.0
        return normalised_X

    word2 = preprocess_Input(word2)
    prediction_scores = loaded_Pruned_model_keras_h5.predict(np.expand_dims(word2, axis=0))
    predicted_index = np.argmax(prediction_scores)
    print("Predicted label: " + classes[predicted_index])
    print("Actual label: " + classes[i])


<h1>Convert Squeeze Net trained model in pytorch to a keras model

In [ ]:
import onnx
print(onnx.__version__)

In [ ]:
import torch
import torch.nn as nn
import keras
import torchvision
from pytorch2keras.converter import pytorch_to_keras

# Load the saved SqueezeNet model
squeezenet = nn.Sequential()
squeezenet.features = torch.nn.Sequential(*(list(torchvision.models.squeezenet1_1(pretrained=True).features.children())))
squeezenet.classifier = torch.nn.Sequential(*(list(torchvision.models.squeezenet1_1(pretrained=True).classifier.children())))

# Convert the model to Keras
input_shape = (3, 224, 224)  # channels first
keras_model = pytorch_to_keras(squeezenet, input_var=torch.zeros((1, *input_shape)))

# Save the Keras model to a file
keras_model.save('squeezenet.h5')


In [ ]:
import torch
import onnx
import onnx2keras

# Load the pre-trained PyTorch model
model = torch.hub.load('pytorch/vision:v0.9.0', 'squeezenet1_0', pretrained=True)

# Export the PyTorch model to ONNX format
dummy_input = torch.randn(1, 3, 224, 224)
onnx_model_path = "squeezenet.onnx"
torch.onnx.export(model, dummy_input, onnx_model_path, verbose=True)

# Convert the ONNX model to Keras format
onnx_model = onnx.load(onnx_model_path)
k_model = onnx2keras.onnx_to_keras(onnx_model,['input.1'])

# Save the Keras model to disk
k_model.save("squeezenet.h5")


In [5]:
import onnx
from onnx2keras import onnx_to_keras

# Load ONNX model
onnx_model = onnx.load('squeezenet.onnx')

# Call the converter (input - is the main model input name, can be different for your model)
k_model = onnx_to_keras(onnx_model, ['input.1'])
     

INFO:onnx2keras:Converter is called.
DEBUG:onnx2keras:List input shapes:
DEBUG:onnx2keras:None
DEBUG:onnx2keras:List inputs:
DEBUG:onnx2keras:Input 0 -> input.1.
DEBUG:onnx2keras:List outputs:
DEBUG:onnx2keras:Output 0 -> 117.
DEBUG:onnx2keras:Gathering weights to dictionary.
DEBUG:onnx2keras:Found weight features.0.weight with shape (96, 3, 7, 7).
DEBUG:onnx2keras:Found weight features.0.bias with shape (96,).
DEBUG:onnx2keras:Found weight features.3.squeeze.weight with shape (16, 96, 1, 1).
DEBUG:onnx2keras:Found weight features.3.squeeze.bias with shape (16,).
DEBUG:onnx2keras:Found weight features.3.expand1x1.weight with shape (64, 16, 1, 1).
DEBUG:onnx2keras:Found weight features.3.expand1x1.bias with shape (64,).
DEBUG:onnx2keras:Found weight features.3.expand3x3.weight with shape (64, 16, 3, 3).
DEBUG:onnx2keras:Found weight features.3.expand3x3.bias with shape (64,).
DEBUG:onnx2keras:Found weight features.4.squeeze.weight with shape (16, 128, 1, 1).
DEBUG:onnx2keras:Found weigh

ValueError: '/features/features.0/Conv_output_0/' is not a valid root scope name. A root scope name has to match the following pattern: ^[A-Za-z0-9.][A-Za-z0-9_.\\/>-]*$

In [ ]:
import onnx

from onnx_tf.backend import prepare

onnx_model = onnx.load("squeezenet.onnx")  # load onnx model
tf_rep = prepare(onnx_model)  # prepare tf representation
tf_rep.export_graph("output_path")  # export the model

In [ ]:
loaded_model_keras = tf.keras.models.load_model("Saved Models For SqueezeNet/0_Saved_Model_With_TF1")
# loaded_model_keras = tf.keras.models.load_model("Saved Models For MobileNet/4-Saved_Model_With_TFWith_Keras_without_the_last_9_layers")

In [ ]:
saved_model_path = "Saved_Model_With_TF1"
tf.saved_model.save(loaded_model_keras, saved_model_path)
loaded_model_keras.save("MobileNetWithoutTheLast6Layers.h5")
print("Saved Model")
saved_model_path = "Saved_Model_With_TFWith_Keras"
tf.keras.models.save_model(loaded_model_keras, saved_model_path)